## CONTEXTUAL FEATURE SELECTION WITH CONDITIONAL STOCHASTIC GATES

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np

# Load dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Reduce to maximum 1000 rows
# X = X[:1000, :]
# y = y[:1000]

# Original column names
column_names = housing.feature_names
feature_names = np.array(column_names)
# Add noise columns
np.random.seed(42)  # For reproducibility

# Gaussian noise
gaussian_noise = np.random.normal(0, 1, size=X.shape[0])

# Uniform noise
uniform_noise = np.random.uniform(-1, 1, size=X.shape[0])

# Cosine function
cosine_values = np.cos(np.linspace(0, 10, X.shape[0]))

# Create a DataFrame from X
df = pd.DataFrame(X, columns=column_names)
df=df.sample(frac=1,random_state=42).reset_index(drop=True)
# Add the noise columns to DataFrame
df['Gaussian_Noise'] = gaussian_noise
df['Uniform_Noise'] = uniform_noise
df['Cosine_Noise'] = cosine_values
df=df.sample(frac=1,random_state=42).reset_index(drop=True)

# Shuffle column locations
np.random.seed(42)  # Ensure reproducibility for column shuffling
shuffled_columns = np.random.permutation(df.columns)
df = df[shuffled_columns]

# Now, df is a DataFrame with shuffled columns and includes the noise features
# You can view the DataFrame as follows:
display(df.head())

# Convert target to a DataFrame and concatenate with features for a complete view
y_df = pd.DataFrame(y, columns=['Target'])
df_full = pd.concat([df, y_df], axis=1)

# If you wish to proceed with splitting and scaling:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset

# Splitting the data (assuming you want to keep DataFrame structure for X)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# X_train_scaled and X_test_scaled are now DataFrames with scaled features and retained column names.
# convert them to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# new column names
feature_names = np.array(X_train_scaled.columns)
# feature_names

,AveOccup,MedInc,Uniform_Noise,Cosine_Noise,AveRooms,HouseAge,Gaussian_Noise,Population,Longitude,AveBedrms,Latitude
0,2.247299,4.5625,0.760274,-0.958842,4.845138,46.0,-0.706843,1872.0,-122.59,1.027611,37.97
1,3.267930,4.5000,-0.134879,0.105413,5.262517,17.0,-1.869742,2415.0,-119.19,1.012179,34.23
2,3.445217,5.2174,0.784740,0.261862,7.306957,5.0,0.116566,3962.0,-117.21,1.078261,33.95
3,2.345733,2.3083,-0.577435,-0.877524,5.485777,20.0,1.198482,1072.0,-122.63,1.262582,38.96
4,2.496000,6.0000,0.833185,0.042673,5.442667,26.0,-0.133279,936.0,-117.24,0.781333,34.15


## Loss Func

In [2]:
import torch
import torch.nn as nn
from torch.distributions import Normal
# Let's define a function to calculate the custom loss
def custom_loss_function(model_output, y_true, mu_d_z, sigma, D, lambda_reg):
    # Calculate MSE loss
    mse_loss = nn.functional.mse_loss(model_output, y_true)
    # Calculate the regularization term
    normal_dist = Normal(torch.zeros_like(mu_d_z), torch.ones_like(mu_d_z) * sigma)
    regularization_term = torch.sum(normal_dist.cdf(mu_d_z)) / D  # normalize by D
    # Combine MSE loss with the regularization term
    total_loss = mse_loss + lambda_reg * regularization_term
    return total_loss
lambda_reg = 0.01
sigma = 1.0  # Given constant sigma

## Model

In [3]:
import torch
from torch import nn
from torch.optim import Adam
from torch.nn.init import xavier_uniform_
    
from torch.distributions.normal import Normal
from torch.autograd import Variable

# Define the Hypernetwork
class C_StochasticGates(nn.Module):
    def __init__(self, input_dim):
        super(C_StochasticGates, self).__init__()
        self.dropout1 = nn.Dropout(0.5)   # Dropout layer with 50% probability
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(input_dim, 128)
        xavier_uniform_(self.fc1.weight)  # Xavier initialization
        self.fc2 = nn.Linear(128, 64)
        xavier_uniform_(self.fc2.weight)  # Xavier initialization
        self.fc3 = nn.Linear(64, 32)
        xavier_uniform_(self.fc3.weight)  # Xavier initialization
        self.fc4 = nn.Linear(32, input_dim)
        xavier_uniform_(self.fc4.weight)  # Xavier initialization
        self.fc5 = nn.Linear(64, 1) # final layer for regression
        xavier_uniform_(self.fc5.weight)  # Xavier initialization
        self.sigma = 1.0  # Given constant sigma
    def forward(self, x):
        """"" hϕ: selection of contextual probabilities """
        x_original = x# Copy the input tensor
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Applying dropout after activation
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)  # Applying dropout after activation
        x = torch.relu(self.fc3(x))
        x = self.dropout1(x)  # Applying dropout after activation
        x = torch.relu(self.fc4(x))
        x = self.dropout1(x)
        
        # Assuming that self.fc4 outputs the logit z which is fed into the sigmoid to obtain mu_z
        mu_z = torch.sigmoid(x)  # Sigmoid to ensure output is a probability

        # stochastic_gates = torch.stack(stochastic_gates, dim=1)
        stochastic_gates = []
        for d in range(mu_z.size(1)):  # Loop over each dimension
            epsilon_d = torch.normal(0, self.sigma, size=(mu_z.size(0),), device=mu_z.device)
            sigma_d = torch.clamp(mu_z[:, d] + epsilon_d, min=0, max=1)
            stochastic_gates.append(sigma_d)

        stochastic_gates = torch.stack(stochastic_gates, dim=1)
        """" Fully connected network for regression parameters yˆ(k)"""
        selected_features = x_original * stochastic_gates
        x = torch.relu(self.fc1(selected_features))
        x = self.dropout1(x)  # Applying dropout after activation
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)  # Applying dropout after activation
        x = self.fc5(x)  # No activation function, suitable for regression
        return x,stochastic_gates
c_stg = C_StochasticGates(input_dim=X_train_scaled.shape[1])
    

## Training

In [5]:
import torch
from torch.optim import Adam
import numpy as np

# Assuming your custom_loss_function is defined somewhere above

def mse_metric(preds, targets):
    return torch.mean((preds - targets) ** 2)

def rmse_metric(preds, targets):
    return torch.sqrt(mse_metric(preds, targets))


In [6]:
def train(model, optimizer, epochs, X_train, y_train, X_val, y_val):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        y_pred, stochastic_gates_pred = model(X_train)
        loss = custom_loss_function(y_pred, y_train, stochastic_gates_pred, sigma=1, D=X_train.shape[1], lambda_reg=0.05)
        loss.backward()
        optimizer.step()

        train_mse = mse_metric(y_pred, y_train).item()
        train_rmse = rmse_metric(y_pred, y_train).item()

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_y_pred, val_stochastic_gates = model(X_val)
            val_loss = custom_loss_function(val_y_pred, y_val, val_stochastic_gates, sigma=1, D=X_val.shape[1], lambda_reg=0.05)
            val_mse = mse_metric(val_y_pred, y_val).item()
            val_rmse = rmse_metric(val_y_pred, y_val).item()
        #"Gates: {stochastic_gates_pred}\n 
        print(f'Epoch {epoch+1},\nTrain: \n  Loss: {loss.item()}, MSE: {train_mse}, RMSE: {train_rmse}, \nVal:\n   Loss: {val_loss.item()}, Val MSE: {val_mse}, Val RMSE: {val_rmse}')
    return stochastic_gates_pred

# Initialize the hypernetwork, prediction network, loss criterion, and optimizer
model = C_StochasticGates(input_dim=X_train_tensor.shape[1])
optimizer = Adam(list(model.parameters()) , lr=0.001)

# Training
epochs = 100  # Adjust as necessary based on convergence and performance
gates = train(model,  optimizer, epochs, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

Epoch 1,
Train: 
  Loss: 568.4468383789062, MSE: 4.833375930786133, RMSE: 2.1984939575195312, 
Val:
   Loss: 144.49537658691406, Val MSE: 4.48626184463501, Val RMSE: 2.118079662322998
Epoch 2,
Train: 
  Loss: 567.5112915039062, MSE: 4.478652000427246, RMSE: 2.1162827014923096, 
Val:
   Loss: 144.12757873535156, Val MSE: 4.1342668533325195, Val RMSE: 2.033289670944214
Epoch 3,
Train: 
  Loss: 567.1289672851562, MSE: 4.129367828369141, RMSE: 2.0320847034454346, 
Val:
   Loss: 143.71194458007812, Val MSE: 3.8135781288146973, Val RMSE: 1.95283842086792
Epoch 4,
Train: 
  Loss: 566.5739135742188, MSE: 3.8911373615264893, RMSE: 1.9725966453552246, 
Val:
   Loss: 143.5850067138672, Val MSE: 3.5433366298675537, Val RMSE: 1.8823752403259277
Epoch 5,
Train: 
  Loss: 566.3935546875, MSE: 3.555668830871582, RMSE: 1.8856481313705444, 
Val:
   Loss: 143.20066833496094, Val MSE: 3.261601686477661, Val RMSE: 1.80599045753479
Epoch 6,
Train: 
  Loss: 565.8513793945312, MSE: 3.425224781036377, RMSE: 1.8

In [7]:
import torch

def r2_score(y_true, y_pred):
    """Compute R-squared score."""
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2.item()

def train(model, optimizer, epochs, X_train, y_train, X_val, y_val):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        y_pred, stochastic_gates_pred = model(X_train)
        loss = custom_loss_function(y_pred, y_train, stochastic_gates_pred, sigma=1, D=X_train.shape[1], lambda_reg=0.05)
        loss.backward()
        optimizer.step()

        train_mse = mse_metric(y_pred, y_train).item()
        train_rmse = rmse_metric(y_pred, y_train).item()
        train_r2 = r2_score(y_train, y_pred)

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_y_pred, val_stochastic_gates = model(X_val)
            val_loss = custom_loss_function(val_y_pred, y_val, val_stochastic_gates, sigma=1, D=X_val.shape[1], lambda_reg=0.05)
            val_mse = mse_metric(val_y_pred, y_val).item()
            val_rmse = rmse_metric(val_y_pred, y_val).item()
            val_r2 = r2_score(y_val, val_y_pred)

        print(f'Epoch {epoch+1},\n'
              f'Train: Loss: {loss.item()}, MSE: {train_mse}, RMSE: {train_rmse}, R^2: {train_r2}\n'
              f'Val: Loss: {val_loss.item()}, Val MSE: {val_mse}, Val RMSE: {val_rmse}, Val R^2: {val_r2}')

    return stochastic_gates_pred

# Note: Ensure custom_loss_function, mse_metric, rmse_metric, and C_StochasticGates are defined
# Also, ensure X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor are properly initialized

# Example initialization (for illustration purposes)
model = C_StochasticGates(input_dim=X_train_tensor.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 100  # Adjust as necessary
gates = train(model, optimizer, epochs, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)


Epoch 1,
Train: Loss: 571.0518188476562, MSE: 7.17909574508667, RMSE: 2.6793835163116455, R^2: -4.370446681976318
Val: Loss: 145.882080078125, Val MSE: 5.757336616516113, Val RMSE: 2.399445056915283, Val R^2: -3.3935413360595703
Epoch 2,
Train: Loss: 570.5059204101562, MSE: 6.826672077178955, RMSE: 2.612790107727051, R^2: -4.106810092926025
Val: Loss: 145.67938232421875, Val MSE: 5.391133785247803, Val RMSE: 2.3218815326690674, Val R^2: -3.114084243774414
Epoch 3,
Train: Loss: 569.3228759765625, MSE: 6.30553674697876, RMSE: 2.511082887649536, R^2: -3.716965675354004
Val: Loss: 145.19595336914062, Val MSE: 5.089291095733643, Val RMSE: 2.2559456825256348, Val R^2: -2.883742332458496
Epoch 4,
Train: Loss: 568.7809448242188, MSE: 5.924668788909912, RMSE: 2.4340641498565674, R^2: -3.432051181793213
Val: Loss: 144.83914184570312, Val MSE: 4.781925678253174, Val RMSE: 2.1867613792419434, Val R^2: -2.6491854190826416
Epoch 5,
Train: Loss: 567.9943237304688, MSE: 5.593095779418945, RMSE: 2.3649

## Comparing with stg

In [ ]:
# import matplotlib.pyplot as plt

# # Define a function to train and evaluate models using STG algorithm
# def train_evaluate_stg(X_train, y_train, X_test, y_test, context_dim):
#     # Instantiate HyperNetwork and PredictionNetwork for STG
#     hypernetwork_stg = HyperNetwork(context_dim, X_train.shape[1] - context_dim)
#     prediction_network_stg = PredictionNetwork(X_train.shape[1] - context_dim)

#     # Train the model
#     stg_losses_train, stg_losses_val = train_model(X_train, y_train, X_test, y_test, 
#                                                     hypernetwork_stg, prediction_network_stg)
#     return stg_losses_train, stg_losses_val

# # Define a function to train and evaluate models using CSTG algorithm
# def train_evaluate_cstg(X_train, y_train, X_test, y_test, context_dim):
#     # Instantiate ConditionalStochasticGates model
#     cstg_model = ConditionalStochasticGates(context_dim, X_train.shape[1] - context_dim)

#     # Train the model
#     cstg_losses_train, cstg_losses_val = train_model(X_train, y_train, X_test, y_test, 
#                                                       cstg_model, prediction_network)
#     return cstg_losses_train, cstg_losses_val

# # Define a function to train the model and return training and validation losses
# def train_model(X_train, y_train, X_test, y_test, hypernetwork, prediction_network):
#     optimizer = optim.Adam(list(hypernetwork.parameters()) + list(prediction_network.parameters()), lr=0.001)
#     loss_fn = nn.MSELoss()

#     num_epochs = 100
#     stg_losses_train, stg_losses_val = [], []

#     for epoch in range(num_epochs):
#         # Training step
#         hypernetwork.train()
#         prediction_network.train()
#         optimizer.zero_grad()

#         # Forward pass
#         gates_train = hypernetwork(X_train[:, :context_dim])
#         selected_features_train = X_train[:, context_dim:] * stochastic_gates(gates_train)
#         predictions_train = prediction_network(selected_features_train)
#         loss_train = loss_fn(predictions_train, y_train)

#         # Backward pass
#         loss_train.backward()
#         optimizer.step()

#         # Validation step
#         with torch.no_grad():
#             hypernetwork.eval()
#             prediction_network.eval()

#             gates_val = hypernetwork(X_test[:, :context_dim])
#             selected_features_val = X_test[:, context_dim:] * stochastic_gates(gates_val)
#             predictions_val = prediction_network(selected_features_val)
#             loss_val = loss_fn(predictions_val, y_test)

#         stg_losses_train.append(loss_train.item())
#         stg_losses_val.append(loss_val.item())

#     return stg_losses_train, stg_losses_val

# # Define the context dimension
# context_dim = 3  # Assuming the first 3 features are used as context

# # Train and evaluate models using STG algorithm
# stg_losses_train, stg_losses_val = train_evaluate_stg(X_train_tensor, y_train_tensor, 
#                                                       X_test_tensor, y_test_tensor, context_dim)

# # Train and evaluate models using CSTG algorithm
# cstg_losses_train, cstg_losses_val = train_evaluate_cstg(X_train_tensor, y_train_tensor, 
#                                                           X_test_tensor, y_test_tensor, context_dim)

# # Plot training and validation losses for STG and CSTG algorithms
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, num_epochs + 1), stg_losses_train, label='STG Train Loss', color='blue')
# plt.plot(range(1, num_epochs + 1), stg_losses_val, label='STG Validation Loss', linestyle='--', color='blue')
# plt.plot(range(1, num_epochs + 1), cstg_losses_train, label='CSTG Train Loss', color='orange')
# plt.plot(range(1, num_epochs + 1), cstg_losses_val, label='CSTG Validation Loss', linestyle='--', color='orange')
# plt.title('Training and Validation Losses')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)
# plt.show()

----

In [27]:

# # Assuming X_train_tensor and X_test_tensor are already defined and properly shaped
# input_dim = X_train_tensor.shape[1]  # Number of features in the input data
# feature_dim = input_dim  # The hypernetwork output and prediction input dimensions must match

# # Initialize the hypernetwork, prediction network, loss criterion, and optimizer
# hypernet = HyperNet(input_dim)
# model = PredNet(feature_dim)
# optimizer = Adam(list(hypernet.parameters()) + list(model.parameters()) , lr=0.01)

# # Training
# epochs = 2  # Adjust as necessary based on convergence and performance
# train(model, hypernet, criterion, optimizer, epochs, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)


In [28]:
# Making predictions with the trained models
def make_inference(model, hypernet, X_test):
    model.eval()
    hypernet.eval()
    
    with torch.no_grad():
        # Get the feature selection probabilities from the hypernetwork
        selection_prob = hypernet(X_test)
        # For inference use the expected values (probabilities) instead of sampling
        selected_features = X_test * selection_prob
        # Get the model predictions
        predictions = model(selected_features)
    return predictions, selection_prob

# Load the test data (replace this with your actual test data)
X_test = X_test_tensor  # Assuming X_test_tensor is your test data

# Make inference
predictions, feature_importance_probabilities = make_inference(model, hypernet, X_test)


# Print the conditional probability of each feature being significant
print("Conditional probabilities of feature significance given other features:")
for i, feature_name in enumerate(feature_names):
    print(f"{feature_name}: {feature_importance_probabilities[:, i].mean().item()}")

Conditional probabilities of feature significance given other features:
AveOccup: 0.44150832295417786
MedInc: 0.5098904967308044
Uniform_Noise: 0.474697470664978
Cosine_Noise: 0.4606289863586426
AveRooms: 0.47779104113578796
HouseAge: 0.461354523897171
Gaussian_Noise: 0.5596177577972412
Population: 0.407277375459671
Longitude: 0.5244668126106262
AveBedrms: 0.4461166560649872
Latitude: 0.5383712649345398


In [29]:
# Create a list of tuples (feature_name, probability)
feature_probabilities = [(feature_name, feature_importance_probabilities[:, i].mean().item()) for i, feature_name in enumerate(feature_names)]

# Sort the list by probability in descending order
sorted_feature_probabilities = sorted(feature_probabilities, key=lambda x: x[1], reverse=True)

# Print the sorted list
print("Conditional probabilities of feature significance given other features:")
for feature, probability in sorted_feature_probabilities:
    print(f"{feature}: {probability}")

Conditional probabilities of feature significance given other features:
Gaussian_Noise: 0.5596177577972412
Latitude: 0.5383712649345398
Longitude: 0.5244668126106262
MedInc: 0.5098904967308044
AveRooms: 0.47779104113578796
Uniform_Noise: 0.474697470664978
HouseAge: 0.461354523897171
Cosine_Noise: 0.4606289863586426
AveBedrms: 0.4461166560649872
AveOccup: 0.44150832295417786
Population: 0.407277375459671


In [5]:
# Calculate the mean feature importance probability across all examples
mean_feature_importance_probabilities = feature_importance_probabilities.mean(dim=0)

# Determine which features have a mean selection probability above the threshold
important_features_indices = mean_feature_importance_probabilities > threshold

print("Mean feature selection probabilities from the hypernetwork (higher means more important):")
print(mean_feature_importance_probabilities)

print("\nFeatures with mean selection probability higher than threshold:")
print(important_features_indices)
# Extract the names of the important features
important_feature_names = [name for i, name in enumerate(feature_names) if important_features_indices[i]]

print("\nImportant feature names:")
print(important_feature_names)


NameError: name 'threshold' is not defined

In [ ]:
# ... (after the training loop)

# Retrieve the final parameters of HyperNet
hypernet_params = hypernet.state_dict()
print("Final parameters of HyperNet:")
for param_name, param in hypernet_params.items():
    print(f"{param_name}:\n{param}")

# Retrieve the final parameters of PredNet
prednet_params = model.state_dict()
print("\nFinal parameters of PredNet:")
for param_name, param in prednet_params.items():
    print(f"{param_name}:\n{param}")


Final parameters of HyperNet:
fc1.weight:
tensor([[ 0.1502, -0.1104, -0.0366,  ...,  0.1718,  0.0637, -0.0697],
        [-0.1804,  0.0159, -0.1112,  ...,  0.1196, -0.0227, -0.0770],
        [ 0.1346,  0.1256,  0.1248,  ..., -0.0737, -0.1655, -0.1717],
        ...,
        [-0.0565,  0.0891,  0.0982,  ..., -0.1358, -0.1359,  0.0080],
        [-0.0074, -0.0703,  0.0896,  ...,  0.2032, -0.2027, -0.1844],
        [-0.0535,  0.0381,  0.0786,  ..., -0.1920, -0.0853, -0.0579]])
fc1.bias:
tensor([ 0.1512,  0.2043,  0.1095, -0.1715, -0.0402, -0.1496, -0.2718,  0.2931,
         0.1333,  0.1599,  0.0045, -0.1857, -0.2328,  0.0870,  0.1531, -0.2914,
         0.0040,  0.2220, -0.0121, -0.1919,  0.0696, -0.0186,  0.0074,  0.0958,
        -0.0544,  0.2858, -0.1110, -0.0218,  0.0621, -0.1147, -0.2755,  0.1071,
         0.2763,  0.1478, -0.2083,  0.2526,  0.0627, -0.0947, -0.2117, -0.2135,
         0.1645, -0.2223,  0.1970, -0.1847, -0.0560, -0.2816, -0.2027, -0.0012,
         0.2529,  0.2172, -0.0118,

In [ ]:
# raise error
raise ValueError("This is a custom error message. You can replace this with an actual error message.")

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
import numpy as np

from torch.nn.init import xavier_uniform_
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal, Bernoulli

class HyperNet(nn.Module):
    def __init__(self, input_dim):
        super(HyperNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(128, input_dim)  # The output should match the number of features
        xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

class PredNet(nn.Module):
    def __init__(self, feature_dim):
        super(PredNet, self).__init__()
        self.fc1 = nn.Linear(feature_dim, 128)
        xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize models (assuming 11 input features)
input_dim = X_train_tensor.shape[1]  # This should be the number of features in your input
feature_dim = input_dim  # This should match the number of input features
hypernet = HyperNet(input_dim)
model = PredNet(feature_dim)

# Rest of the initialization and training code...
criterion = nn.MSELoss()
optimizer = Adam(list(model.parameters()) + list(hypernet.parameters()), lr=0.001)

hypernet.train()
optimizer.zero_grad()

# Forward pass through hypernetwork to get selection probabilities
selection_prob = hypernet(X_train)
# Sample from Bernoulli distribution to get feature mask
selection_mask = torch.bernoulli(selection_prob)
# Apply mask to input features
selected_features = X_train * selection_mask

# Make predictions with masked features
y_pred = model(selected_features)
loss = criterion(y_pred, y_train)

# Backpropagation
loss.backward()
optimizer.step()

# Validation phase
model.eval()
hypernet.eval()
# with torch.no_grad():
#     val_selection_prob = hypernet(X_val)
#     val_selection_mask = torch.bernoulli(val_selection_prob)
#     val_selected_features = X_val * val_selection_mask
#     val_y_pred = model(val_selected_features)
#     val_loss = criterion(val_y_pred, y_val)

# print(f'Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')

# Model initialization


# Training
# epochs = 100 # Adjust as necessary
# train(model, hypernet, criterion, optimizer, epochs, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)


TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [ ]:

# # Define the Hypernetwork
# class HyperNet(nn.Module):
#     def __init__(self, input_dim, feature_dim):
#         super(HyperNet, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128) # Adjust the size as necessary
#         self.fc2 = nn.Linear(128, feature_dim)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = self.sigmoid(self.fc2(x))
#         return x


# class HyperNet(nn.Module):
#     def __init__(self, input_dim, feature_dim):
#         super(HyperNet, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128)
#         # Initialize weights using Xavier initialization
#         xavier_uniform_(self.fc1.weight)
#         self.fc2 = nn.Linear(128, feature_dim)
#         xavier_uniform_(self.fc2.weight)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         # Final layer with sigmoid to ensure outputs are between 0 and 1
#         x = self.sigmoid(self.fc2(x))
#         return x

class HyperNet(nn.Module):
    def __init__(self, input_dim, feature_dim):
        super(HyperNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(128, feature_dim)  # feature_dim should match the number of input features
        xavier_uniform_(self.fc2.weight)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Define the Prediction Network
# class PredNet(nn.Module):
#     def __init__(self, feature_dim):
#         super(PredNet, self).__init__()
#         self.fc1 = nn.Linear(feature_dim, 128) # Adjust the size as necessary
#         self.fc2 = nn.Linear(128, 1) # Assuming a single output
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

class PredNet(nn.Module):
    def __init__(self, feature_dim):
        super(PredNet, self).__init__()
        self.fc1 = nn.Linear(feature_dim, 128)  # feature_dim should match the number of input features
        self.fc2 = nn.Linear(128, 1)

# Initialize models (assuming 11 input features)
input_dim = 11
feature_dim = input_dim  # Make sure this matches the number of input features
hypernet = HyperNet(input_dim, feature_dim)
model = PredNet(feature_dim)
def train(model, hypernet, criterion, optimizer, epochs, X_train, y_train, X_val, y_val):
    for epoch in range(epochs):
        model.train()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16512x128 and 11x128)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch.distributions.bernoulli import Bernoulli


In [ ]:
class Hypernetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Hypernetwork, self).__init__()
        # Define the architecture of the hypernetwork.
        self.fc1 = nn.Linear(input_dim, 128) # Input layer
        self.fc2 = nn.Linear(128, output_dim) # Output layer mapping to Bernoulli parameters

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.fc1(x)) # Activation function for non-linearity
        x = torch.sigmoid(self.fc2(x)) # Sigmoid to ensure output is in [0,1], representing probabilities
        return x


In [ ]:
class PredictionNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PredictionNetwork, self).__init__()
        # Define the architecture of the prediction network.
        self.fc1 = nn.Linear(input_dim, 128) # Input layer
        self.fc2 = nn.Linear(128, output_dim) # Output layer for the response variable

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.fc1(x)) # Activation function for non-linearity
        x = self.fc2(x) # Output layer
        return x


In [ ]:
def train(model, hypernet, criterion, optimizer, data_loader, epochs=10):
    for epoch in range(epochs):
        for x, z, y in data_loader: # Assuming x is the feature, z is the context, y is the target
            optimizer.zero_grad()
            
            # Generate probabilities from the Hypernetwork
            probs = hypernet(z)
            
            # Sample from Bernoulli to get the feature selection mask
            m = Bernoulli(probs)
            mask = m.sample()
            
            # Apply mask and predict
            x_masked = x * mask
            predictions = model(x_masked)
            
            # Compute loss and backpropagate
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
